In [1]:
%config HistoryManager.enabled = False

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import torch
from interpreter import interpreter
import re

In [ ]:
model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model_to_use = AutoModelForCausalLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
).cuda()

In [ ]:
# def extract_code_block(text: str) -> str:
#     """
#     Extracts the first full markdown code block from the given text.
    
#     Returns:
#         The full code block string, including triple backticks and language tag,
#         or the original text if no code block is found.
#     """
#     match = re.search(r"```[\w+-]*\n.*?```", text, re.DOTALL)
#     if match:
#         return match.group(0).strip()
#     return text
# Plug into OpenInterpreter
interpreter.llm.context_window      = 2048
interpreter.llm.max_tokens          = 512
interpreter.llm.supports_vision     = False
interpreter.llm.supports_functions  = True
interpreter.auto_run = True
def extract_code_block(markdown: str):
    """
    Parses the first code block in a markdown string.

    Returns:
        A tuple of (language, code), or None if no code block is found.
    """
    match = re.search(r"```([\w+-]*)\n(.*?)```", markdown, re.DOTALL)
    if match:
        language = match.group(1) or "text"
        code = match.group(2)
        return code
    return markdown

def hf_completions(messages, model, stream, max_tokens, tools):
    print(tools)
    prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True,  tokenize=False)
    
    # Set up streaming
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Use a separate thread for generation to enable streaming
    from threading import Thread
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=max_tokens)
    thread = Thread(target=model_to_use.generate, kwargs=generation_kwargs)
    thread.start()

    # Stream the tokens as they are generated
    for token in streamer:
        yield {"choices": [{"delta": {"content": token}}]}
    
    # Tokenize and generate
    # inputs = tokenizer(prompt, return_tensors="pt").to(model_to_use.device)
    # outputs = model_to_use.generate(
    #     **inputs,
    #     streamer=streamer,
    #     max_new_tokens=max_tokens,
    # )

    # Stream tokens as they're generated
    # for token in streamer:
    #     yield {"choices": [{"delta": {"content": token}}]}



     # 3) Extract just the generated part
    # gen_ids = outputs[0][ inputs["input_ids"].shape[-1] : ]
    # text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    
        
    # #processed = extract_code_block(text)
    # yield {"choices": [{"delta": {"content": text}}]}
    # yield {"choices": [{"delta": {"content": processed}}]}
    # print(processed)
    #  # Stream in chunks of 10-50 characters
    # chunk_size = 50
    # global all_words
    # for i in range(0, len(processed), chunk_size):
    #     chunk = processed[i:i + chunk_size]
    #     all_words.append(chunk)
    #     yield {"choices": [{"delta": {"content": chunk}}]}
    

interpreter.llm.completions         = hf_completions
# Now test a simple prompt
interpreter.messages = []
messages = interpreter.chat("Figure out who the user is in this machine. Use python")

In [ ]:
tools_[0]

In [ ]:
def extract_code_block(markdown: str):
    """
    Parses the first code block in a markdown string.

    Returns:
        A tuple of (language, code), or None if no code block is found.
    """
    match = re.search(r"```([\w+-]*)\n(.*?)```", markdown, re.DOTALL)
    if match:
        language = match.group(1) or "text"
        code = match.group(2)
        return code
    return markdown

print(extract_code_block(messages[0]['content']))

In [ ]:
interpreter.messages

In [ ]:
print(interpreter.messages[2]['content'])

In [ ]:
interpreter.messages[1]['content']

In [ ]:
for m in interpreter.messages:
    print(m['role'])
    print(m['type'])
    print(m['content'])
    print('------')

In [ ]:
print(interpreter.messages[1:3])

In [ ]:
print(interpreter.system_message)

In [ ]:
interpreter.llm

In [ ]:
import re

m = re.search(r"```bash\n(.+?)```", last, re.DOTALL)
bash_snippet = m.group(1).strip()
print(bash_snippet)
# 4) Execute it
result = interpreter.computer.run("bash", bash_snippet)
result

In [ ]:
bash_snippet

In [ ]:
import re
from transformers import TextIteratorStreamer

def hf_completions(messages, model, stream, max_tokens):
    prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    
    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model_to_use.device)
    outputs = model_to_use.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Extract just the generated part
    gen_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    
    # Stream by words for better performance
    words = text.split(' ')
    for i, word in enumerate(words):
        content = word if i == 0 else ' ' + word
        yield {"choices": [{"delta": {"content": content}}]}

# Configure Open Interpreter
interpreter.llm.context_window = 2048
interpreter.llm.max_tokens = 512
interpreter.llm.supports_vision = False
interpreter.llm.supports_functions = False

# IMPORTANT: Add execution instructions since supports_functions = False
interpreter.llm.execution_instructions = """To execute code on the user's machine, write a markdown code block. Specify the language after the ```. You will receive the output. Use any programming language.

Example:
```python
print("Hello World")
```"""

# Set auto_run
interpreter.auto_run = True

# Set the custom completions function
interpreter.llm.completions = hf_completions

# Clear messages and test
interpreter.messages = []
print("Starting chat...")
messages = interpreter.chat("Figure out who the user is in this machine. Use python")

# Print the actual messages to debug
print("\n=== MESSAGES DEBUG ===")
for i, msg in enumerate(interpreter.messages):
    print(f"Message {i}:")
    print(f"  Role: {msg.get('role', 'N/A')}")
    print(f"  Type: {msg.get('type', 'N/A')}")
    if 'content' in msg:
        content = msg['content']
        if len(content) > 200:
            print(f"  Content: {content[:200]}...")
        else:
            print(f"  Content: {content}")
    print()

In [ ]:
interpreter.llm.max_iterations